# Capstone project
--------------------------------------------

## by Tal Meyerstein

* This notebook will be used for the Capstone project
* It will contain both Python code and markdown

### 1. First - import all the necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup # library for scraping HTML
print('Libraries imported.')

Libraries imported.


### 2. Second - send an http request to the Toronto wiki page and extract the HTML from the response

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url).text
results

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat"

Extracting the html content from the response.
The HTML openning tag is at the 17th character of the result string,
and the closing tag ends 1 character before the end of the string:

In [3]:
html_doc = results[16:-1]

Calling the Beautiful Soup library for scraping HTML:

In [4]:
soup = BeautifulSoup(html_doc, 'html.parser')
print(soup.prettify())

<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateForma

### 3. Finaly - I use Beautiful Soup's HTML parser to store the PostalCode rows in a DataFrame

In [5]:
# populate a Dataframe with the html table contents
toranto_df = pd.DataFrame(columns=['PostalCode','Borough', 'Neighborhood'])
# loop over the list of <tr> tags. skip the header <tr>
for tr_tag in soup.table.find_all('tr')[1:]: 
    PostalCode = tr_tag.find_all('td')[0].string # read the PostalCode from the first <td> of the row
    
    # read the Borough from the second <td>. look for it under the <a> tag and if not exist then directly under the td
    Borough = (tr_tag.find_all('td')[1].a.string if tr_tag.find_all('td')[1].a else tr_tag.find_all('td')[1].string)
    
    # read the Neighborhood from the third <td>. look for it under the <a> tag and if not exist then directly under the td
    Neighborhood = (tr_tag.find_all('td')[2].a.string if tr_tag.find_all('td')[2].a else tr_tag.find_all('td')[2].string)    
    
    # trim all the whitespace chars
    Neighborhood = Neighborhood.strip()
    
    # if Neighborhood is 'Not assigned' then set it same as the Borough
    Neighborhood = (Neighborhood if Neighborhood != 'Not assigned' else Borough)
    
    if (Borough != 'Not assigned'): # ignore rows with a borough that is not assigned
        if PostalCode in toranto_df['PostalCode'].values: # check if PostalCode already in DataFrame 
            # If already in df then concatenate the neighborhood to the existing Neigborhood cell
            toranto_df.loc[toranto_df['PostalCode']==PostalCode, 'Neighborhood'] = toranto_df.loc[toranto_df['PostalCode']==PostalCode, 'Neighborhood'] + ', ' + Neighborhood
        else: # else append a new row to the DataFrame
            toranto_df = toranto_df.append({'PostalCode' : PostalCode,'Borough' : Borough, 'Neighborhood' : Neighborhood},ignore_index=True)
toranto_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [6]:
toranto_df.shape[0]

103

In [7]:
# load the Postal codes coordinates from a CSV file
PostalCodes_Coordinates = pd.read_csv('Geospatial_Coordinates.csv')
PostalCodes_Coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [8]:
# rename the postal code column so we'll be able to implictely merge the 2 DFs
PostalCodes_Coordinates.rename({'Postal Code':'PostalCode'}, axis=1, inplace=True)
# merge based on PostalCode coulumn
toranto_df = pd.merge(toranto_df, PostalCodes_Coordinates)
print(toranto_df.shape)
toranto_df

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [9]:
# getting the coordinates of Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toranto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toranto_df['Latitude'], toranto_df['Longitude'], toranto_df['Borough'], toranto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
CLIENT_ID = 'MIU3ZKAVB0NPUPANUFW024CALVKDILZ4MR5SQYIU25HY4E0V' # my Foursquare ID
CLIENT_SECRET = 'CAQFF0BBSKGI55UVWNQJ2FQNF3DF3NRVOQM02Z4EFKKMNOGP' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

radius = 500
LIMIT = 100

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: MIU3ZKAVB0NPUPANUFW024CALVKDILZ4MR5SQYIU25HY4E0V
CLIENT_SECRET:CAQFF0BBSKGI55UVWNQJ2FQNF3DF3NRVOQM02Z4EFKKMNOGP


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=toranto_df['Neighborhood'],
                                   latitudes=toranto_df['Latitude'],
                                   longitudes=toranto_df['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [14]:
# Note that around 3 neighborhoods zero venues were found, so we're down from 103 to 100.
print(toronto_venues['Neighborhood'].value_counts().shape)

(100,)


In [15]:
count_venues = toronto_venues.groupby('Neighborhood').count()
count_venues.drop(['Neighborhood Latitude','Neighborhood Longitude','Venue Latitude', 'Venue Longitude', 'Venue Category'],axis=1,inplace=True)
count_venues.rename({'Venue':'Venues Count'},axis=1,inplace=True)
count_venues.reset_index(inplace=True)
count_venues

,Neighborhood,Venues Count
0,"Adelaide, King, Richmond",100
1,Agincourt,4
2,"Agincourt North, L'Amoreaux East, Milliken, St...",2
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",9
4,"Alderwood, Long Branch",9
5,"Bathurst Manor, Downsview North, Wilson Heights",17
6,Bayview Village,4
7,"Bedford Park, Lawrence Manor East",24
8,Berczy Park,58
9,"Birch Cliff, Cliffside West",4


In [16]:
toranto_count_df = pd.merge(toranto_df,count_venues)
toranto_count_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Venues Count
0,M3A,North York,Parkwoods,43.753259,-79.329656,3
1,M4A,North York,Victoria Village,43.725882,-79.315572,5
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,49
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,11
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,42
5,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1
6,M3B,North York,Don Mills North,43.745906,-79.352188,5
7,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,13
8,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,100
9,M6B,North York,Glencairn,43.709577,-79.445073,4


In [17]:
# lets again create map of Toronto with the neighborhoods. but this time with colors marking how many venues per restuarant
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, countv in zip(toranto_count_df['Latitude'], toranto_count_df['Longitude'], toranto_count_df['Borough'], toranto_count_df['Neighborhood'], toranto_count_df['Venues Count']):
    label = '{}, {}, venues count:{}'.format(neighborhood, borough,countv)
    label = folium.Popup(label, parse_html=True)
    CircleColor = 'green'
    if countv < 30:
        if countv < 10:
            CircleColor = 'red'
        else:
            CircleColor = 'orange'
            
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=CircleColor,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [35]:
# I see that the vast majority of the neighborhoods that are outside central Toranto have little venues so I decided to drop them
central_toranto_count_df = toranto_count_df[toranto_count_df['Borough'].str.contains('Toronto')]
central_toranto_count_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Venues Count
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,49
8,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,100
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,100
18,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
19,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,58
23,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,86
24,M6G,Downtown Toronto,Christie,43.669542,-79.422564,15
29,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,100
30,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,18
35,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,100


In [19]:
# Also dropping the neighborhoods with less than 10 venous
central_toranto_count_df.drop([89,81,66,65,60,59,18],axis=0,inplace=True)
central_toranto_count_df

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Venues Count
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,49
8,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,100
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,100
19,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,58
23,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,86
24,M6G,Downtown Toronto,Christie,43.669542,-79.422564,15
29,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,100
30,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,18
35,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,100
36,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,61


In [20]:
central_toranto_count_df.shape

(31, 6)

In [21]:
# Once again drawing the toronto collored neighborhoods map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, countv in zip(central_toranto_count_df['Latitude'], central_toranto_count_df['Longitude'], central_toranto_count_df['Borough'], central_toranto_count_df['Neighborhood'], central_toranto_count_df['Venues Count']):
    label = '{}, {}, venues count:{}'.format(neighborhood, borough,countv)
    label = folium.Popup(label, parse_html=True)
    CircleColor = 'green'
    if countv < 30:
        if countv < 10:
            CircleColor = 'red'
        else:
            CircleColor = 'orange'
            
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=CircleColor,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Now it looks good!

#### Lets filter out all the venues that do not belong to any of the Central Toronto neighborhoods:

In [36]:
mask = np.isin(toronto_venues['Neighborhood'].values,central_toranto_count_df['Neighborhood'].values)
central_toronto_venues = toronto_venues[mask]
central_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
8,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
10,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
11,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
12,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


#### Let's find out how many unique categories can be curated from all the returned venues

In [37]:
print('There are {} uniques categories.'.format(len(central_toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


## 3. Analyze Each Neighborhood

In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['AAANeighborhood'] = central_toronto_venues['Neighborhood'] # Pre-fixing the Neighborhood column with "AAA" to distinuish from the "Neighborhood" Category

# move neighborhood column to the first column and rename it back to neighborhood
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.rename({'AAANeighborhood':'Neighborhood Name'},axis=1, inplace=True)
toronto_onehot.head()

,Neighborhood Name,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
8,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [39]:
toronto_onehot.shape

(1690, 237)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighborhood Name').mean().reset_index()
toronto_grouped

,Neighborhood Name,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.000000,0.03000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.00,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.060000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.030000,0.000,0.000000,0.000000,0.010000,0.000000,0.010000

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood Name']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.03
3         Cheese Shop  0.03
4  Seafood Restaurant  0.03


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.11
1            Café  0.11
2     Coffee Shop  0.11
3      Restaurant  0.05
4             Bar  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0          Smoke Shop  0.07
1          Restaurant  0.07
2  Light Rail Station  0.07
3             Brewery  0.07
4              Garden  0.07


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.15
1   Airport Service  0.15
2  Ai

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood Name']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Restaurant,Burger Joint,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Restaurant,Café,Farmers Market,Seafood Restaurant,Beer Bar,Steakhouse
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Coffee Shop,Café,Burrito Place,Stadium,Caribbean Restaurant,Bar,Restaurant,Furniture / Home Store,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Fast Food Restaurant,Garden Center,Pizza Place,Park,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery,Farmers Market
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique,Boat or Ferry,Sculpture Garden


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [48]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = central_toranto_count_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [50]:
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Venues Count,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,49,2,Coffee Shop,Café,Pub,Bakery,Park,Mexican Restaurant,Theater,Gym / Fitness Center,Breakfast Spot,Restaurant
8,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,100,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Restaurant,Fast Food Restaurant,Middle Eastern Restaurant,Juice Bar,Burger Joint,Tea Room
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,100,2,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Italian Restaurant,Park,Cosmetics Shop,Cocktail Bar,Clothing Store
18,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,2,Neighborhood,Health Food Store,Coffee Shop,Pub,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
19,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,58,2,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Restaurant,Café,Farmers Market,Seafood Restaurant,Beer Bar,Steakhouse
23,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,86,2,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Bubble Tea Shop,Spa,Chinese Restaurant,Juice Bar,Middle Eastern Restaurant
24,M6G,Downtown Toronto,Christie,43.669542,-79.422564,15,2,Grocery Store,Café,Park,Convenience Store,Baby Store,Diner,Italian Restaurant,Coffee Shop,Restaurant,Nightclub
29,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,100,2,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Restaurant,Burger Joint,Bakery
30,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,18,2,Pharmacy,Supermarket,Bakery,Discount Store,Park,Music Venue,Liquor Store,Middle Eastern Restaurant,Café,Bar
35,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,100,2,Coffee Shop,Aquarium,Hotel,Pizza Place,Café,Italian Restaurant,Scenic Lookout,Brewery,Restaurant,Bakery


Finally, let's visualize the resulting clusters

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, I will examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2,5] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,Venues Count,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,Roselawn,1,Garden,Yoga Studio,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster 2

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2,5] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,Venues Count,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Lawrence Park,4,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
66,"Forest Hill North, Forest Hill West",4,Jewelry Store,Park,Sushi Restaurant,Trail,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store
89,Rosedale,4,Park,Playground,Trail,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 3

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2,5] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,Venues Count,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Harbourfront, Regent Park",49,Coffee Shop,Café,Pub,Bakery,Park,Mexican Restaurant,Theater,Gym / Fitness Center,Breakfast Spot,Restaurant
8,"Ryerson, Garden District",100,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Restaurant,Fast Food Restaurant,Middle Eastern Restaurant,Juice Bar,Burger Joint,Tea Room
14,St. James Town,100,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Italian Restaurant,Park,Cosmetics Shop,Cocktail Bar,Clothing Store
18,The Beaches,4,Neighborhood,Health Food Store,Coffee Shop,Pub,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
19,Berczy Park,58,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Restaurant,Café,Farmers Market,Seafood Restaurant,Beer Bar,Steakhouse
23,Central Bay Street,86,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Bubble Tea Shop,Spa,Chinese Restaurant,Juice Bar,Middle Eastern Restaurant
24,Christie,15,Grocery Store,Café,Park,Convenience Store,Baby Store,Diner,Italian Restaurant,Coffee Shop,Restaurant,Nightclub
29,"Adelaide, King, Richmond",100,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Restaurant,Burger Joint,Bakery
30,"Dovercourt Village, Dufferin",18,Pharmacy,Supermarket,Bakery,Discount Store,Park,Music Venue,Liquor Store,Middle Eastern Restaurant,Café,Bar
35,"Harbourfront East, Toronto Islands, Union Station",100,Coffee Shop,Aquarium,Hotel,Pizza Place,Café,Italian Restaurant,Scenic Lookout,Brewery,Restaurant,Bakery


## 6. Conclusions

Based on the above examinations I conclude that Central Toronto is more homogeneous than Manhattan. The vast majorities of its neighborhoods contain a mixture of Restaurants, Coffee shops and Bars (Cluster 2), while a few of its neighborhoods on its outskirts (Cluster 1) seem to be of resedential character - contain Parks and Trails. There is 1 neighbborhood which i will consider an outlier since only 1 Foursquare Venue was found in it and the algorithem has classified it as Cluster 0.